<a href="https://colab.research.google.com/github/Jorge-89/Analisis-Exploratorio-de-Datos-en-Python/blob/main/%F0%9F%8D%B7_Predicting_Wine_Quality_%E2%9C%8588_Accuracy%F0%9F%94%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'red-wine-quality-cortez-et-al-2009:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4458%2F8204%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240711%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240711T005044Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D41a9b3b23e251c3052f724b28c404d6bd16f51b5e072c69467427014382aa6a33a8f2a401c06ca5d78ec450d2bdfacaa3b6029f38f0172583f162e6ac889628ab2a68cb57bd18d2f8f99b49b6043733d15c69511243c9c2d4237ca00a1d8750585e97c6f2129134b297fb22b045b11504860ab0bb467038994cbc8e9c5a9e4a2a184bd45b4bbd642d35986f28bdb5e6b905dd582a0e31c6481c8da98c8deec5b52c9d9bad25e9812b2489caec6b9f6a23fcbe27b92fac66efe3b26814276b5df7f880ed931db3e2b876817e108749ca341ddb1e81300f02d599d065b2ab10e5614e524585fb7c856b657740146cb0825f68217dce3b1af433c7514d6ed9f47f9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>✨ Red Wine Quality Prediction ✨</b></div>

<h3 align="center" style="font-size: 35px; color: #800080; font-family: Georgia;">
    <span style="color: #008080;"> Author:</span>
    <span style="color: black;">Kumod Sharma .📄🖋️</span>
</h3>

<img src="https://user-images.githubusercontent.com/47216809/86511259-ef33e380-be14-11ea-8ea3-3a3cc4ff5ce8.jpg" style='width: 1400px; height: 450px;'>

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b> 🎬 Introduction 🎬</b></div>

<div style="border-radius:10px;border:black solid;padding: 15px;background-color:white;font-size:110%;text-align:left">
<div style="font-family:Georgia;background-color:'#DEB887'; padding:30px; font-size:17px">

   

<h3 align="left"><font color=purple>📝 Project Objective:</font></h3><br>
    
1. Develop a <b>predictive model</b> using machine learning algorithms to accurately assess and <b>predict the quality of red wines</b> based on various chemical properties and attributes.<br>

2. Evaluate and <b>compare the performance of different machine learning techniques</b> to determine the most effective approach for red wine quality prediction, providing insights for potential applications in the wine industry.<br>
    
3. <b>Dataset Link:-</b> <a href="https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009">Click to get the Dataset</a><br>
    
    

    
    

<h3 align="left"><font color=purple>🌟 Business Understanding:</font></h3><br>
    
1. <b>Enhanced Product Quality:</b> Accurate red wine quality prediction will lead to improved product quality and consistency, enhancing the winery's reputation and customer satisfaction.<br>
    
2. <b>Cost Optimization:</b> Optimal resource allocation and reduced wastage through predictive modeling will result in cost savings for wineries, improving overall operational efficiency.<br>

3. <b>Market Competitiveness:</b> Consistent production of high-quality red wines will give wineries a competitive advantage, allowing them to stand out in the market and attract more customers.<br>
    </div>
    </div>

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b> ⚙️ Project Content ⚙️</b></div>

<div style="border-radius:10px;border:black solid;padding: 15px;background-color:white;font-size:110%;text-align:left">
<div style="font-family:Georgia;background-color:'#DEB887'; padding:30px; font-size:16px">

<h3 align="left"><font color=purple>📊 Table of Contents:</font></h3><br>
    
    
1. <b>📚 Importing Libraries:</b> - To perform <b>Data Manipulation,Visualization & Model Building.</b><br>
    
    
2. <b>⏳ Loading Dataset:</b> - Load the dataset into a <b>suitable data structure using pandas.</b><br>

    
3. <b>🧠 Basic Understaning of Data:</b> - Generate basic informations about the data.<br>
    
    
4. <b>🧹 Data Cleaning:</b> - To <b>clean, transform, and restructure</b> the data in order to make it suitable for analysis.<br>

    
5. <b>📊 Exploatory Data Analysis:</b> -  To  identify <b>trends, patterns, and relationships</b> among the variabels.<br>

    
6. <b>📈 Feature Selection:</b> -  To identify <b>most relevant features</b> for model building.<br>
    
    
7. <b>⚙️ Data Preprocessing:</b> - To transform data for creating more accurate & robust model.<br>
    
    
8. <b>🎯 Model building:</b>- To build <b>predictive models</b>, using various algorithms.<br>
    
    
9. <b>⚡️ Model evaluation:</b> - To analyze the Model performance using metrics.<br>
    
    
10. <b>🍀 Stacking Model:</b>- To develop a stacked model using the top performing models.<br>


11. <b>🎈 Conclusion:</b> - Conclude the project by summarizing the <b>key findings.</b><br>
    
</div>

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b> 📚 Importing Libraries 📚</b></div>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set(style="darkgrid",font_scale=1.5)
sns.set(rc={"axes.facecolor":"#FFFAF0","figure.facecolor":"#FFFAF0"})
sns.set_context("poster",font_scale = .7)

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from scipy import stats
from scipy import special
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

from imblearn.over_sampling import SMOTE

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>⏳ Loading Datset ⏳</b></div>

In [ ]:
df = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
df.head().style.set_properties(**{'background-color': '#E9F6E2','color': 'black','border-color': '#8b8c8c'})

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>🧠 Basic Understanding of Data 🧠</b></div>

### 1. Cheking Dimension of Dataset.

In [ ]:
df.shape

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=brown>📊 Inference:</font></h3>

* There are total **1599 Records/Rows** in the dataset.
* There are total **12 Features/columns** in the dataset.

### 2. Generating Basic Information about Data.

In [ ]:
df.info(verbose=False)

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=brown>📊 Inference:</font></h3>

* All the **features** present is in  the dataset is **numerical.**
* No **Categorical Features** present in the dataset.

### 3. Computing Total No. of Missing Values & Percentage of Missing values.

In [ ]:
null_df = df.isnull().sum().to_frame().rename(columns={0:"Total No. of Missing Values"})
null_df["% of Missing values"] = round(100*null_df["Total No. of Missing Values"]/len(df),2)
null_df.sort_values(by="% of Missing values",ascending=False)

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="left"><font color=brown>📊 Inference:</font></h3>

* **None of the features** is hvaing **missing values.**
* So we can say the dataset will be more reliable for prediction wine quality.

---

### 4. Checking Presence of Duplicate Records in Dataset.

In [ ]:
print("Is there any Duplicate Records => ",df.duplicated().any())
print("-"*42)
print("Total Duplicate Records present is =>",df[df.duplicated()==True].shape[0])

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The  **first output** is **True** which indicates  that is **presenece of Duplicate Records.**
* The **second output** is **240** which indicates that there is total **240 Duplicate Records.**
* Duplicate vlaues can lead to **Data Integrity issues** so it's better to **drop these records.**

----

### 5. Dropping Duplicate Records.

In [ ]:
df.drop_duplicates(inplace=True)

----

### 6. Performing Descriptive Statistical Analysis on Numerical Features.

In [ ]:
df.describe().T.style.set_properties(**{'background-color': '#E9F6E2','color': 'black','border-color': '#8b8c8c'})

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

1. The **minimum wine quality is 3** and the **maximum wine quality is 8.**
2. The **average alchol** a red wine holds according to the data is **10.4**.
3. There a **huge difference** between **average total sulfur dioxide** and **maximum total sulfur dioxide**.

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>📊 Exploratory Data Analysis.📊</b></div>

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 1. Visualizing the Target Variable </div>

In [ ]:
plt.figure(figsize=(13.7,6))
z = df["quality"].value_counts()
sns.barplot(x=z.index, y=z.values, order=z.index, palette=["#11264e","#6faea4","#FEE08B","#D4A1E7","#E7A1A1"])
plt.title("Target Feature Distributions",fontweight="black",size=25,pad=15)
for index,value in enumerate(z.values):
    plt.text(index,value,value, ha="center", va="bottom",fontweight="black")

plt.tight_layout()
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* Most of the wines are having quality of **5 or 6**.
* We can clearly observe a **class-imbalance** in the target feature.
* To **overcome** the class-imbalance we can use techniques like:-
    1. **`SMOTE (synthetic minority oversampling technique).`**
    
    2. **`Stratified K-Folds cross-validation.`**
   
    
* **Note:** In this project I have use **Stratified K-Fold Cross-Validaton.**

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 2. Visualizing "Fixed Acidity" Attribute. </div>

In [ ]:
def numerical_plot(column):
    plt.figure(figsize=(13.5,10))
    plt.subplot(2,1,1)
    sns.boxplot(x="quality",y=column, data=df, palette=["#FFA07A","#D4A1E7","#FFC0CB","#87CEFA","#F08080","#D8BFD8"])
    plt.title(f"{column.title()} vs Quality Analysis",fontweight="black",size=25,pad=10,)

    plt.subplot(2,1,2)
    sns.histplot(x=column,kde=True,hue="quality",data=df, palette="Set2")
    skew = df[column].skew()
    plt.title(f"Skewness of {column.title()} Feature is: {round(skew,3)}",fontweight="black",size=20,pad=10)
    plt.tight_layout()
    plt.show()

In [ ]:
numerical_plot("fixed acidity")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **fixed acidity** is having **almost a symmetric distribution** but the distribution is **little right skewed** with a skewness value of **0.941**.
* Skewness can lead to several implications like **model performance**, **hypothesis testing** and it returns **biased estimation.**
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution.**

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 3. Visualizing "Volatile Acidity" Attribute. </div>

In [ ]:
numerical_plot("volatile acidity")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Volatile Acidity** is having **almost a symmetric distribution** but the distribution is **right skewed** with a skewness value of **0.729**..
* Skewness can lead to several implications like **model performance**, **hypothesis testing** and it returns **biased estimation.**
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution.**

----

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 4. Visualizing "Citric Acid" Attribute. </div>

In [ ]:
numerical_plot("citric acid")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Citric Acid** is having a distribution of **right skewed** with a skewness value of **0.313**.
* Although the **skewness is low** but still we try to bring the **skewness close to 0.**
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution.**

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 5. Visualizing "Residual Sugar" Attribute. </div>

In [ ]:
numerical_plot("residual sugar")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Residual Sugar** is having **almost a symmetric distribution** but the distribution is ** highly right skewed** with a skewness value of **4.548**.
* The distrbution is **highly right skewed** because of presence of **outliers.**   
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution** and bring **skewness close to 0.**

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 6. Visualizing "Chlorides" Attribute. </div>

In [ ]:
numerical_plot("chlorides")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Chlorides** is having **almost a symmetric distribution** but the distribution is **highly right skewed** with a skewness value of **5.502**..
* The distrbution is **highly right skewed** because of presence of **outliers.**   
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution** and bring **skewness close to 0.**

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 7. Visualizing "Free Sulfur Dioxide" Attribute. </div>

In [ ]:
numerical_plot("free sulfur dioxide")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Free Sulfur Dioxide** is having a **highly right skewed distribution** with a skewness value of **1.227**..
* The distrbution is **highly right skewed** because of presence of **outliers.**   
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution** and bring **skewness close to 0.**

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 8. Visualizing "Total Sulfur Dioxide" Attribute. </div>

In [ ]:
numerical_plot("total sulfur dioxide")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Total Sulfur Dioxide** is having a **highly right skewed distribution** with a skewness value of **1.54**.
* The distrbution is **highly right skewed** because of presence of **outliers.**   
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution** and bring **skewness close to 0.**

----

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 9. Visualizing "Density" Attribute. </div>

In [ ]:
numerical_plot("density")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Density** is having a perfect **Noraml Distribution** because the **skewness is close to 0**.
* So we **don't** have to use aany transformation techniques on this feature.

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 10. Visualizing "pH" Attribute. </div>

In [ ]:
numerical_plot("pH")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **pH** is having a **Normal Distribution** with a skewness value of **0.232**..
* But still the **tails** is little **right skewed** because of presence of **outliers.**
* So we will use **transformation techniques** to deal with those outliers.

----

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 11. Visualizing "Sulphates" Attribute. </div>

In [ ]:
numerical_plot("sulphates")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Sulphates** is having **almost a symmetric distribution** but the distribution is **highly right skewed** with a skewness value of **2.407**.
* The distrbution is **highly right skewed** because of presence of **outliers.**   
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution** and bring **skewness close to 0.**

----

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 12. Visualizing "Alcohol" Attribute. </div>

In [ ]:
numerical_plot("alcohol")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The feature **Alcohol** is having **Asymmetric Distribution** and the distribution is **highly right skewed** with a skewness value of **0.86**.
* The distrbution is **highly right skewed** because of presence of **outliers.**   
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution** and bring **skewness close to 0.**

---

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 13. Visualizing Correlation Among the Independent Atrributes. </div>

In [ ]:
columns = df.columns.tolist()
columns.remove("quality")  ##Quality is having discrete values that's why it's not the correct way to check its importane.

corr = df[columns].corr()

plt.figure(figsize=(13.5,10))

sns.heatmap(corr,fmt=".2g",annot=True ,cmap='YlOrRd_r')
plt.title("Correlation all Independet Features",fontweight="black",size=25,pad=20)
plt.tight_layout()
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* Many **features** are having **high correlation** with the other features:-
    1. **Fixed Acidity** is having high correlation with **citic acid** and **pH** and vice-versa.
    2. **Volatile Acidity** is having high correlation with **citic acid** adn vice-versa.
    3. **Free Sulfur Dioxide**  is having high correlation with **total  sulfur dioxide**, **pH**, **Sulphates** and vice-versa.
    4. **Density** is having high correaltion with **fixed acidity**, **alcohol** and vice-versa.
  

* **Note:**
    * We **can't drop these** correlted features because these features **helps algorithms to create pattern** for prediction.

----

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b> 📈 Statistical Analysis - Feature Importance.📈 </b></div>

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 23px;border-style: solid;border-color: dark green;"> 1. Performing ANOVA Test to Analyze the Features Importance in Wine Quality. </div>

In [ ]:
f_scores = {}
p_values = {}

for column in columns:
    f_score, p_value = stats.f_oneway(df[column],df["quality"])

    f_scores[column] = f_score
    p_values[column] = p_value

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 2. Visualizing the F_Score of ANOVA Test of Features. </div>

In [ ]:
plt.figure(figsize=(15,6))
keys = list(f_scores.keys())
values = list(f_scores.values())

sns.barplot(x=keys, y=values, palette="pastel")
plt.title("Anova-Test F_scores Comparison",fontweight="black",size=20,pad=15)
plt.xticks(rotation=90)

for index,value in enumerate(values):
    plt.text(index,value,int(value), ha="center", va="bottom",fontweight="black",size=15)
plt.show()

## <div style="text-align: left; background-color:#CFD9F1 ; font-family: Trebuchet MS; color: #0A3399; padding: 15px; line-height:1;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;border-style: solid;border-color: dark green;"> 3. Comparing F_Score and P_value of ANOVA Test.  </div>

In [ ]:
test_df = pd.DataFrame({"Features":keys,"F_Score":values})
test_df["P_value"] = list(p_values.values())

In [ ]:
test_df

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The **following features showed statistically significant associations with wine quality:**
    * **Volatile Acidity.**
    * **Citric Acid.**
    * **Chlorides.**
    * **Density.**
    * **pH.**
    * **Sulphates.**
    * **Alcohol.**
    
    
* The **following features did not show statistically significant associations with wine quality.**
    * **Fixed Acidity.**
    * **Residual Sugar.**
    * **Free Sulfur Dioxide.**
    * **Total Sulfur Dioxide.**

    
    
* **Note:**
    * **Expect Residual Sugar** feature the other **3 features** were having **high correlation with** other independent features.
    * So we **can't drop those 3 features** but if required we **can drop the Residual Sugar** feature.

----

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>⚙️ Data Preprocessing ⚙️</b></div>

### 1. Computing Skewness of Each Numeircal Attributes.

In [ ]:
new_df  = df.copy()
columns = df.columns.tolist()
columns.remove("quality")

In [ ]:
skew_df = df[columns].skew().to_frame().rename(columns={0:"Skewness"})
skew_df

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* Except **Density**, **pH**, **citric acid** all the other features are having **high skewness.**
* Skewness can lead to several implications like **model performance**, **hypothesis testing** and it returns **biased estimation.**
* So we will use **tranformation techniques** to transform thesse feature to have a **symmetric distribution.**

----

### 2. Performing Skewness Transformation Analysis using Different Transformation Techniques.

In [ ]:
columns = df.columns.tolist()
columns.remove("quality")
skewness_transformation = {}

for col in columns:
    transformed_log = np.log(df[col])                        # Log Transformation
    transformed_boxcox = special.boxcox1p(df[col], 0.15)     # Box-Cox Transformation with lambda=0.15
    transformed_inverse = 1 / df[col]                        # Inverse Transformation
    transformed_yeojohnson, _ = stats.yeojohnson(df[col])    # Yeo-Johnson Transformation
    transformed_cbrt = np.cbrt(df[col])                      # Cube Root Transformation

    # Create a dictionary for the skewness values of each transformation
    transformation_skewness = {
        "Log Transformation": stats.skew(transformed_log),
        "Box-Cox Transformation": stats.skew(transformed_boxcox),
        "Inverse Transformation": stats.skew(transformed_inverse),
        "Yeo Johnson Transformation": stats.skew(transformed_yeojohnson),
        "Cube Root Transformation": stats.skew(transformed_cbrt)}

    # Store the transformation skewness values for the column
    skewness_transformation[col] = transformation_skewness


In [ ]:
result_df = pd.DataFrame.from_dict(skewness_transformation, orient='index')
result_df = pd.concat([skew_df["Skewness"], result_df], axis=1)
result_df

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* Out of all the transformation the **Yeo Johnson Transformation** has given the **best results** by reducing the skewness.
* **Inverse transformation** performation is also good on some of the features.
* But we will use **Yeo Johnson Transformation**  to achieve th **Normal Distribution.**

----

### 3. Applying Yeo - Johnson Transformation on Independent variables.

In [ ]:
for col in columns:
    transformed_col,_ = stats.yeojohnson(df[col])
    df[col] = transformed_col

In [ ]:
df.sample(5)

---

### 4. Comparining Distribution of Features Before & After Transformations.

In [ ]:
x=1
y=2


plt.figure(figsize=(25,60))
for col in columns:
    plt.subplot(11,2,x)
    sns.histplot(new_df[col],kde=True,color="purple")
    plt.title(f"Distribution of {col} Before Transformation",fontweight="black",size=25,pad=10)
    x+=2

    plt.subplot(11,2,y)
    sns.histplot(df[col],kde=True, color="orange")
    plt.title(f"Distribution of {col} After Transformation",fontweight="black",size=25,pad=10)
    y+=2

    plt.tight_layout()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* We can clearly observe the **changes** their Distrbutions.
* Most of the **features** has achieved **Normal Distribution.**

----

### 4. Splitting Target Variable into two Groups.

In [ ]:
df["quality"].unique()

In [ ]:
#Condition of Splitting: If quality > 6.5 => "good" ELSE => "bad"

bin_edges = [0,6.5,10]
group_names = ["Bad","Good"]

df["quality"] = pd.cut(df["quality"], bins=bin_edges, labels=group_names)
df["quality"].unique()

---

### 5. Visualizing the New Distribution of Target Variable.

In [ ]:
plt.figure(figsize=(17,6))
plt.subplot(1,2,1)
quality_counts = df["quality"].value_counts()
sns.barplot(x=quality_counts.index, y=quality_counts.values,palette=["#1d7874","#8B0000"])
plt.title("Wine Quality Value Counts",fontweight="black",size=20,pad=20)
for i, v in enumerate(quality_counts.values):
    plt.text(i, v, v,ha="center", fontweight='black', fontsize=18)

plt.subplot(1,2,2)
plt.pie(quality_counts, labels=["Bad","Good"], autopct="%.2f%%", textprops={"fontweight":"black","size":15},
        colors = ["#1d7874","#AC1F29"],explode=[0,0.1],startangle=90)
center_circle = plt.Circle((0, 0), 0.3, fc='white')
fig = plt.gcf()
fig.gca().add_artist(center_circle)
plt.title("Wine Quality Values Distrbution" ,fontweight="black",size=20,pad=10)
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* We can still observe **class - imbalance**.
* The dataset is having more **Bad Quality Wines** and very **low Good quality wines.**

---

### 6. Encoding Target Variable.

In [ ]:
df["quality"] = df["quality"].replace({"Bad":0,"Good":1})

In [ ]:
df.sample(5)

----

### 7. Splitting Dataset using StratifiedKFold Method.

In [ ]:
df2 = df.drop(columns="quality")
n_splits = 5  # Set the number of folds
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
for train_index, test_index in stratified_kfold.split(df2, df["quality"]):
    x_train, x_test = df2.iloc[train_index], df2.iloc[test_index]
    y_train, y_test = df["quality"].iloc[train_index], df["quality"].iloc[test_index]

In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The data is **splitted** in **two sets** one for Model Training and another for Model performance Testing.
* We will use **Train Dataset** to train the model and **Test Dataset** to evaluate the performance of the model.

----

### 8. Computing Frequeny of Unique Values in Y_Train & Y_Test.

In [ ]:
unique_df = y_train.value_counts().to_frame().rename(columns={"quality":"Y_train Target frequency"})
unique_df["Y_test target Freequency"] = y_test.value_counts()
unique_df

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* Both the categories in **target variable** is splitter in such a way that **model learns and create pattern** for both the categories easily.

----

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>🎯 Model Creation using DecisionTree 🎯</b></div>

### 1. Performing Grid-Search with cross-validation to find the best Parameters for the Model.

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
param_grid = {"max_depth":[3,4,5,6,7,8,9,10],
              "min_samples_split":[2,3,4,5,6,7,8],
              "min_samples_leaf":[1,2,3,4,5,6,7,8],
              "criterion":["gini","entropy"],
              "splitter":["best","random"],
              "max_features":["sqrt",None],
              "random_state":[0,42]}

In [ ]:
grid_search = GridSearchCV(dtree, param_grid, cv=5, n_jobs=-1)

grid_search.fit(x_train,y_train)

---

### 2. Fetching the Best Parameters for DecisionTree Model.

In [ ]:
best_parameters = grid_search.best_params_

print("Best Parameters for DecisionTree Model is:\n")
best_parameters

---

### 3. Creating DecisionTree Model Using Best Parameters.

In [ ]:
dtree = DecisionTreeClassifier(**best_parameters)

dtree.fit(x_train,y_train)

---

### 4. Computing Model Accuracy.

In [ ]:
y_train_pred = dtree.predict(x_train)
y_test_pred = dtree.predict(x_test)

print("Accuracy Score of Model on Training Data is =>",round(accuracy_score(y_train,y_train_pred)*100,2),"%")
print("Accuracy Score of Model on Testing Data  is =>",round(accuracy_score(y_test,y_test_pred)*100,2),"%")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The model has obtained **88 % accuracy** on training Dataset and **86 % accuracy** on testing dataset.
* So therre's **No underfitting or Overfitting** in the model.
* The model is having a kind of **best fitting.**

---

### 5. Model Evaluation using Different Metric Values.

In [ ]:
print("F1 Score of the Model is =>",f1_score(y_test,y_test_pred, average="weighted"))
print("Recall Score of the Model is =>",recall_score(y_test,y_test_pred, average="weighted"))
print("Precision Score of the Model is =>",precision_score(y_test,y_test_pred, average="weighted"))

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* We can observe that **recall, precision, and F1 score are approxiamately same**, it means that our **model is achieving perfect balance between** correctly identifying **positive samples (recall)** and minimizing **false positives (precision).**
    
    
* The **high values** for **F1 score, recall score, and precision score**, all of which are **more than 0.8.**
* These **metrics suggest** that the **model achieves good accuracy** in predicting the **positive class.**

---

### 6. Finding Importance of Features in DecisionTreeClassifier.

In [ ]:
imp_df = pd.DataFrame({"Feature Name":x_train.columns,
                       "Importance":dtree.feature_importances_})

In [ ]:
features = imp_df.sort_values(by="Importance",ascending=False)

plt.figure(figsize=(12,7))
sns.barplot(x="Importance", y="Feature Name", data=features, palette="plasma")
plt.title("Feature Importance in the Model Prediction", fontweight="black", size=20, pad=20)
plt.yticks(size=12)
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The **key factors** that significantly influence the **quality of wine** are:-
    * **`Alcohol`**, **`Sulphates`**, and **`Volatile Acidity`**.
    
    
* The **minimal impact** of features on the **wine quality** are:-
    * **`free sulfur dioxide`**, **`pH`**, and **`Residual Sugar`**

----

### 7. SHAP Summary Plot: Explaining Model Predictions with Feature Importance.

In [ ]:
import shap
explainer = shap.TreeExplainer(dtree)
shap_values = explainer.shap_values(x_test)

plt.title("Feature Importance and Effects on Predictions",fontweight="black",pad=20,size=18)
shap.summary_plot(shap_values[1], x_test.values, feature_names = x_test.columns,plot_size=(14,8))

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The **red color represents high feature values**, indicating that the **feature positively contributes** for **increasing the prediction value.**
* The **blue color represents low feature values,** indicating that the **feature negatively contributes** for **decreasing the prediction value.**

---

### 8. Model Evaluation using Confusion Matrix.

In [ ]:
cm = confusion_matrix(y_test,y_test_pred)

plt.figure(figsize=(15,6))
sns.heatmap(data=cm, linewidth=.5, annot=True, fmt="g", cmap="Set1")
plt.title("Model Evaluation using Confusion Matrix",fontsize=20,pad=20,fontweight="black")
plt.ylabel("Actual Labels")
plt.xlabel("Predicted Labels")
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* **Strong True Positive Rate:** The model achieved a high number of true positive predictions, indicating its ability to correctly identify positive cases. This suggests that the model is effective in accurately classifying the desired outcome.

    
* **Need of Improvement in False Negative Rate:** The presence of a relatively high number of false negatives suggests that the model may have missed identifying some actual positive cases. This indicates a need for further refinement to enhance the model's ability to capture all positive cases.</div>

---

### 9. Model Evaluation: ROC Curve and Area Under the Curve (AUC)

In [ ]:
y_pred_proba = dtree.predict_proba(x_test)[:][:,1]

df_actual_predicted = pd.concat([pd.DataFrame(np.array(y_test), columns=["y_actual"])])
df_actual_predicted.index = y_test.index


fpr, tpr, thresholds = roc_curve(df_actual_predicted["y_actual"], y_pred_proba)
auc = roc_auc_score(df_actual_predicted["y_actual"], y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}",color="green")
plt.plot([0, 1], [0, 1], linestyle="--", color="black")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve",pad=20,fontweight="black")
plt.legend()
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

1. An **AUC (Area Under the Curve) value of 0.77** suggests that the model has **strong discriminative power.**
2. This suggests that the model has a **high ability to distinguish between positive and negative instances**, indicating its effectiveness in making accurate predictions.
3. The **model has a relatively high probability** of ranking a randomly selected positive instance higher than a randomly selected negative instance.

---

<a id="1"></a>
# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>🎯 Model Creation using RandomForest 🎯</b></div>

### 1. Performing Grid-Search with cross-validation to find the best Parameters for the Model.

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
param_grid = {"max_depth":[3,4,5,6,7,8],
              "min_samples_split":[3,4,5,6,7,8],
              "min_samples_leaf":[3,4,5,6,7,8],
              "n_estimators": [50,70,90,100],
              "criterion":["gini","entropy"]}

In [ ]:
grid_search = GridSearchCV(rfc, param_grid, cv=5, n_jobs=-1)

grid_search.fit(x_train,y_train)

---

### 2. Fetching the Best Parameters for RandomForest Model.

In [ ]:
best_parameters = grid_search.best_params_

print("Best Parameters for RandomForest Model is:\n\n")
best_parameters

---

### 3. Creating RandomForest Model Using Best Parameters.

In [ ]:
rfc = RandomForestClassifier(**best_parameters)

rfc.fit(x_train,y_train)

----

### 4. Computing Model Accuracy.

In [ ]:
y_train_pred = rfc.predict(x_train)
y_test_pred  = rfc.predict(x_test)

print("Accuracy Score of Model on Training Data is =>",round(accuracy_score(y_train,y_train_pred)*100,2),"%")
print("Accuracy Score of Model on Testing Data  is =>",round(accuracy_score(y_test,y_test_pred)*100,2),"%")

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The model has obtained **90 % accuracy** on training Dataset and **87 % accuracy** on testing dataset.
* So there's **No underfitting or Overfitting** in the model.
* The model is having a kind of **best fitting.**

---

### 5. Model Evaluation using Different Metric Values.

In [ ]:
print("F1 Score of the Model is =>",f1_score(y_test,y_test_pred,average="weighted"))
print("Recall Score of the Model is =>",recall_score(y_test,y_test_pred,average="weighted"))
print("Precision Score of the Model is =>",precision_score(y_test,y_test_pred,average="weighted"))

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* We can observe that **recall, precision, and F1 score are approxiamately same**, it means that our **model is achieving perfect balance between** correctly identifying **positive samples (recall)** and minimizing **false positives (precision).**
    
    
* The **high values** for **F1 score, recall score, and precision score**, all of which are **more than 0.8.**
* These **metrics suggest** that the **model achieves good accuracy** in predicting the **positive class.**

---

### 6. Finding Importance of Features in RandomForest Model.

In [ ]:
imp_df = pd.DataFrame({"Feature Name":x_train.columns,
                       "Importance":rfc.feature_importances_})

In [ ]:
features = imp_df.sort_values(by="Importance",ascending=False)

plt.figure(figsize=(12,7))
sns.barplot(x="Importance", y="Feature Name", data=features, palette="plasma")
plt.title("Feature Importance in the Model Prediction", fontweight="black", size=20, pad=20)
plt.yticks(size=12)
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The **key factors** that significantly influence the **deactivation of customers banking facilities** are:-
    * **`Alcohol`**, **`Sulphates`**, and **`Volatilee Acidity`**.
    
    
* The **minimal impact** of features on the **deactivation of customers' banking facilities** are:-
    * **`Density`**, **`Free Sulfur Dioxide`**, and **`pH`**.

----

### 7. SHAP Summary Plot: Explaining Model Predictions with Feature Importance.

In [ ]:
import shap
explainer = shap.TreeExplainer(rfc)
shap_values = explainer.shap_values(x_test)

plt.title("Feature Importance and Effects on Predictions",fontweight="black",pad=20,size=18)
shap.summary_plot(shap_values[1], x_test.values, feature_names = x_test.columns,plot_size=(14,8))

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* The **red color represents high feature values**, indicating that the **feature positively contributes** for **increasing the prediction value.**
* The **blue color represents low feature values,** indicating that the **feature negatively contributes** for **decreasing the prediction value.**

---

### 8. Model Evaluation using Confusion Matrix.

In [ ]:
cm = confusion_matrix(y_test,y_test_pred)

plt.figure(figsize=(15,6))
sns.heatmap(data=cm, linewidth=.5, annot=True, fmt="g", cmap="Set1")
plt.title("Model Evaluation using Confusion Matrix",fontsize=20,pad=20,fontweight="black")
plt.ylabel("Actual Labels")
plt.xlabel("Predicted Labels")
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

* **Strong True Positive Rate:** The model achieved a high number of true positive predictions, indicating its ability to correctly identify positive cases. This suggests that the model is effective in accurately classifying the desired outcome.

    
* **Need of Improvement in False Negative Rate:** The presence of a relatively high number of false negatives suggests that the model may have missed identifying some actual positive cases. This indicates a need for further refinement to enhance the model's ability to capture all positive cases.</div>

---

### 9. Model Evaluation: ROC Curve and Area Under the Curve (AUC)

In [ ]:
y_pred_proba = rfc.predict_proba(x_test)[:][:,1]

df_actual_predicted = pd.concat([pd.DataFrame(np.array(y_test), columns=["y_actual"])])
df_actual_predicted.index = y_test.index


fpr, tpr, thresholds = roc_curve(df_actual_predicted["y_actual"], y_pred_proba)
auc = roc_auc_score(df_actual_predicted["y_actual"], y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}",color="green")
plt.plot([0, 1], [0, 1], linestyle="--", color="black")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve",pad=20,fontweight="black")
plt.legend()
plt.show()

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Inference:</font></h3>

    
1. An **AUC (Area Under the Curve) value of 0.83** suggests that the model has **strong discriminative power.**
2. This suggests that the model has a **high ability to distinguish between positive and negative instances**, indicating its effectiveness in making accurate predictions.
3. The **model has a relatively high probability** of ranking a randomly selected positive instance higher than a randomly selected negative instance.

---

# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>🎈 Conclusion 🎈</b></div>

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Key-Findings:</font></h3>

    
* The **key factors** that significantly influence the **wine quality** are **Alcohol**, **Sulphates** and **Volatilee Acidity.**
    

*  The **minimal impact** of features on the **wine quality** are **Free Sulfur Dioxide** and **pH**.

    
* **High Training and Testing Accuracies:** Both the model achieved a high accuracy score near to 90% on the training data, indicating a good fit to the training instances. Additionally, the model's accuracy score near to 87% on the testing data suggests its ability to generalize well to unseen instances.
    
    
* **High F1 Score, Recall, and Precision:** The model achieved high F1 score, recall, and precision values, all **more than 0.8.** This indicates that the model has a strong ability to correctly identify positive cases while minimizing false positives and maximizing true positives.


* **High AUC value more than 0.8**, states that the model demonstrates a reasonably good discriminatory power. It suggests that the model is able to distinguish between positive and negative instances with a relatively high degree of accuracy.
    
    
* **Overall Model Performance:** The model demonstrates strong performance across multiple evaluation metrics, indicating its effectiveness in making accurate predictions and capturing the desired outcomes.
   

---

# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b>💡 Recommendations 💡</b></div>

<div style="border-radius:10px; border:#808080 solid; padding: 15px; background-color: ##F0E68C ; font-size:100%; text-align:left">

<h3 align="lefAt"><font color=brown>💬 Key-Suggestions:</font></h3>

* **Key Quality Drivers:** Prioritize **Alcohol, Sulphates, and Volatile Acidity** as they significantly influence wine quality.

    
* **Quality Control:** Implement stringent **quality control measures** to maintain desired levels of the key factors during production.

    
* **Varietal-specific Approach:** Tailor the **winemaking process** to suit each grape variety's unique characteristics and quality requirements.

    
   
* **Technology and Expertise:** Invest in modern winemaking technology and **employ experienced winemakers** to ensure precise management of quality factors.

    
    
* **Continuous Improvement:** Continuously monitor **customer feedback and wine ratings** to identify areas for improvement and enhance overall product quality.

---

# <div style="padding:20px;color:white;margin:0;font-size:35px;font-family:Verdana;text-align:center;display:fill;border-radius:5px;background-color:#254E58;overflow:hidden"><b> 🏁 THE END 🏁</b></div>

<div style="background-color:#FFD700; font-family: Georgia; color:black; font-size: 130%; text-align:center; border: 3px solid #000000; border-radius: 20px; padding: 20px;">
    <p style="margin: 0; text-align:center;">🔴 <b>If you liked this Notebook, please do upvote.</b> 🔴</p><br>
    <p style="margin: 0; text-align:center;">🔴 <b>If you have any questions, feel free to comment!</b> 🔴</p><br>
    <p style="margin: 0; text-align:center;"><strong> 🌼 Best Wishes!  🌼</strong></p><br>
</div>

<center> <img src="https://img.freepik.com/free-vector/thank-you-placard-concept-illustration_114360-13436.jpg?w=2000" style='width: 500px; height: 300px;'>

----